In [1]:
import json 
import networkx as nx
  

from conjecture.consolidated_functions import *
from conjecture.all_valid_orderings import *
from conjecture.optimization_functions import *
from iterative_heuristic.modified_etf import Mod_ETF
from iterative_heuristic.consolidated import *
from iterative_heuristic.approximate_speeds import *
from makespan_energy.construct_graph_util import *
from makespan_energy.visualization_util import *
from trace_data.graph_list_from_trace import *
import networkx as nx
import numpy as np
import math
import plotly.express as px
import pandas as pd 

In [2]:
# Construct DAGs

file_list = ['montage-chameleon-2mass-05d-001.json',
'montage-chameleon-2mass-10d-001.json',
'montage-chameleon-2mass-15d-001.json',
'montage-chameleon-2mass-20d-001.json',
'montage-chameleon-dss-05d-001.json',
'montage-chameleon-dss-10d-001.json',
'montage-chameleon-dss-15d-001.json',
'montage-chameleon-dss-20d-001.json']
    
G_list = []

for file in file_list:
    
    # Opening JSON file 
    f = open('pegasus-traces-master/montage/chameleon-cloud/' + file) 

    # returns JSON object as  
    # a dictionary 
    data = json.load(f) 

    taskname_to_id = {}
    id_count = 0

    G = nx.DiGraph()

    for task in data['workflow']['jobs']:

        child_id = id_count
        id_count += 1
        taskname_to_id[task['name']] = child_id

        if not G.has_node(child_id):
                G.add_node(child_id)

        for parent in task['parents']:
            if parent in taskname_to_id:

                parent_id = taskname_to_id[parent]

            else:

                parent_id = id_count
                id_count += 1
                taskname_to_id[parent] = parent_id


            if not G.has_edge(parent_id, child_id):
                    G.add_edge(parent_id, child_id)
    print(len(G.nodes))

    
   
    G_list.append(G)
    
    # Closing file 
    f.close() 

1738
4846
7117
9805
58
472
2122
6448


In [3]:
G_list[0]

In [ ]:
mrt = True
tie_breaking_rule = 2
plot = False
num = 5
x = []
y = []
lbs= []
ubs = []
 
num_machines = 50
naives = []
heur = []
for G in G_list[:num]:
    
    ratio = []
    num_tasks = len(G.nodes)
    
    w = [1 for _ in range(num_tasks)]
    s = [1 for _ in range(num_tasks)]

    naive, true, order =iterative_and_naive_heuristic_no_ratio(num_machines, w, G, verbose=False)
    _, _, obj_opt, _ = get_objective_single_ordering(True, G, w, order, plot=plot, compare=False)
    d = get_optimal_schedule(mrt, G, num_machines, w, plot, verbose=False)
    # if solver was successful
    # and d["objective_task_scaling"] != 10000000
    #if obj_opt != 10000000 and d["objective_task_scaling"] != 10000000:
        
    x.append(num_tasks)
    heur.append(true)
    naives.append(naive)
    y.append(obj_opt)
    lbs.append(d["objective_task_scaling"])
    ub = obj_ub_precise = 2 * sum(psize_to_speed(ub_lst_graph(G, order)))
    ubs.append(ub)
 

[[58, 28, 2, 579, 637, 245, 881, 1371, 157, 373, 821, 880, 1295, 1377, 90, 380, 773, 977, 1438, 316, 298, 951, 1383, 302, 1020, 394, 789, 1426, 1565, 383, 494, 495, 496, 546, 1074, 1075, 1125, 1653, 1654, 1704, 577, 1735, 1736], [59, 29, 3, 580, 652, 235, 748, 1391, 177, 460, 701, 933, 1344, 1389, 95, 381, 774, 986, 1507, 317, 378, 991, 1477, 303, 1030, 229, 1002, 1434, 1566, 720, 497, 547, 1076, 1126, 1655, 1705, 1156, 578, 1737], [75, 30, 4, 590, 654, 260, 798, 1502, 213, 468, 810, 940, 1356, 1486, 96, 104, 746, 1027, 1516, 318, 492, 993, 1487, 304, 1395, 385, 1010, 1526, 1567, 722, 498, 548, 1077, 1127, 1656, 1706, 1734, 1157], [596, 49, 5, 602, 658, 261, 890, 1326, 214, 477, 927, 972, 1386, 1500, 101, 228, 796, 1038, 1517, 319, 87, 711, 1493, 123, 1440, 418, 784, 1532, 1568, 730, 499, 549, 1078, 1128, 1657, 1707], [646, 50, 8, 610, 1178, 265, 891, 1363, 215, 153, 779, 987, 1278, 1506, 102, 277, 856, 1049, 1518, 320, 333, 958, 1374, 168, 1447, 419, 791, 1534, 195, 731, 500, 550, 107

[27, 28, 70.66666666666666, 28, 28, 28, 29, 30, 28, 29, 25, 28, 29, 29, 28, 30, 29, 30, 28, 30, 26, 68.66666666666666, 28, 27, 27, 69.66666666666666, 27, 26, 71.66666666666666, 29, 29, 26, 26, 30, 26, 30, 30, 28, 68.66666666666666, 28, 26, 27, 68.66666666666666, 69.66666666666666, 28, 26, 27, 27, 27, 29, 29, 26, 28, 68.66666666666666, 29, 27, 30, 29, 72.66666666666666, 30, 72.66666666666666, 28, 27, 72.66666666666666, 26, 26, 27, 29, 72.66666666666666, 26, 30, 29, 27, 25, 26, 30, 27, 28, 27, 29, 11, 11, 6, 53.666666666666664, 11, 11, 5, 10, 11, 16, 58.666666666666664, 5, 11, 2, 11, 16, 16, 15, 2, 47.666666666666664, 16, 16, 16, 49.666666666666664, 15, 16, 16, 16, 5, 16, 15, 16, 16, 16, 5, 16, 16, 5, 47.666666666666664, 16, 15, 16, 16, 7, 7, 58.666666666666664, 58.666666666666664, 5, 5, 58.666666666666664, 16, 16, 16, 16, 16, 15, 16, 58.666666666666664, 15, 11, 58.666666666666664, 10, 10, 16, 22, 22, 22, 22, 22, 21, 21, 22, 22, 21, 21, 22, 17, 64.66666666666666, 17, 15, 25, 25, 15, 25, 

-


In [ ]:
print("x is \n", x)
print("y is \n", y)
print("lbs are \n",lbs)
print("ubs are \n", ubs)
print("naives are \n",naives)
print("heurs are \m",heur)

In [ ]:
print(len(y))
print(len(x))

In [ ]:
# naives = [572.8281617409056, 1331.3128540244006, 2051.988186148974, 4908.957570715966, 4427.097315671543, 10722.551392522024, 7706.784325022744, 18780.723094156718, 11887.994968277473, 29080.41842969494, 16967.104097800042, 41618.012251501, 22950.791107151243, 56400.18395313568, 29836.001750405976, 73423.87928867387, 37619.11087992854, 92685.47311048009, 46306.79788927975, 114191.64481211497, 55896.00853253448, 137939.34014765336]
# heurs = [351.14748449053826, 698.6688417479015, 1181.833119651485, 2392.931559303018, 2515.1051913091133, 5128.200522698374, 4320.182955770601, 8876.07384841392, 6602.766119158128, 13562.092685406415, 9411.33279943706, 19356.644767630198, 12670.852536835615, 26103.094518568214, 16404.64052815484, 33831.316526414616, 20688.501817054886, 42685.15172746671, 25668.437288627654, 52905.78716201741, 30587.45634664135, 63200.60308232752]
# y = [300.05921, 586.18851, 10000000, 1637.77108, 1520.32995, 10000000, 2333.07674, 10000000, 3254.04157, 6424.66253, 4271.75539, 10000000, 5377.87606, 10000000, 6565.71298, 12977.19203, 7829.99982, 10000000, 9169.60916, 10000000, 10571.28621, 20904.74224]
ratios1 = [n/h for n,h in zip(naives, heur)]
print(ratios1)
ratios2 = [h/true for h, true in zip(heur, y)]
print(ratios2)
ratios3 = [n/true for n, true in zip(naives, y)]
print(ratios3)

In [ ]:
x = [52, 82, 104, 164, 156, 246, 208, 328, 260, 410, 312, 492, 364, 574, 416, 656, 468, 738, 520, 820, 572, 902]


In [ ]:
import matplotlib.pyplot as plt
import math

fig = plt.figure()
ax = fig.add_axes([0,0,1,1])


#print(ub_ratio)
#plt.scatter(obj_lb_lst[:,0], obj_lb_lst[:,1], label='lower bound', color="g", linewidths=0.1)


# plt.scatter(x, y, s=5, label='opt/lb', color='b')
plt.scatter(x[:num], ratios1, label='Naive/Heur', color='k')
plt.scatter(x[:num], ratios2, label='Heur/Optimal', color='r')
plt.scatter(x[:num], ratios3, label='Naive/Optimal', color='b')
#plt.scatter(x, naives, label='Naives', color='g')
plt.xlabel('Number of tasks')
plt.ylabel('Ratios')
plt.legend()
plt.show()

In [ ]:
import csv
with open("1000genome_naive_heur_3_machines.csv", 'w+') as f:
    writer = csv.writer(f)
    writer.writerow(["task_size", "optimal", "naive", "heuristic", "lowerbound", "upperbound"])
    writer.writerows(zip(x, y, naives, heurs, lbs, ubs))